In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
bc = pd.read_csv("breast.csv")

In [3]:
#bc.info()

In [4]:
bc.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
bc=bc.drop('id',axis=1)

In [6]:
bc = bc.drop('Unnamed: 32', axis=1)

In [7]:
from sklearn import preprocessing  
label_encoder = preprocessing.LabelEncoder()  
bc['diagnosis']= label_encoder.fit_transform(bc['diagnosis'])
bc['diagnosis'].unique()



array([1, 0])

In [8]:
#Heat Map
#heat map is so messy to find corelation
#plt.figure(figsize = (15,10))
#sns.heatmap(bc.corr(), annot = True, cmap = 'RdPu')
#plt.figure(figsize = (2,10))
#sns.heatmap(bc.corr()[['diagnosis']], annot = True, cmap = 'RdPu')


In [9]:
X = bc.drop(['diagnosis'], axis = 1)
Y = bc['diagnosis']

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [11]:
#before PCA we need tom scale the data set
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))
x_train_scaler = scaler.fit_transform(x_train)
x_test_scaler= scaler.transform(x_test)


In [12]:
#now to reduce the extensive variable range of 30, we are using PCA method
#because during heat map its messy to educe variable
from sklearn.decomposition import PCA
obj = PCA(n_components=.70)
obj.fit(x_train_scaler)

PCA(n_components=0.7)

In [13]:
x_train_PCA = obj.transform(x_train_scaler)
x_test_PCA = obj.transform(x_test_scaler)

In [14]:
from sklearn.linear_model import LogisticRegression
# solver = 'lbfgs', C = higher the better, max_iter = 1e7, penalty = 'l2'
logit_model = LogisticRegression(solver='lbfgs', C = 1e4, max_iter=1e3, penalty = 'l2')

In [15]:
logit_model.fit(x_train_PCA, y_train)
logit_model.score(x_test_PCA, y_test)

0.9473684210526315

In [16]:
#prediction
y_pred = logit_model.predict(x_test_PCA)

In [25]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9473684210526315


In [23]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.99      0.93      0.96        76
           1       0.88      0.97      0.93        38

    accuracy                           0.95       114
   macro avg       0.93      0.95      0.94       114
weighted avg       0.95      0.95      0.95       114



In [26]:
# SVM
from sklearn.svm import SVC, SVR
svc_model= SVC()
svc_model = SVC(C=100.0, kernel='linear',gamma='auto')
svc_model.fit(x_train_PCA, y_train)
svc_model.score(x_test_PCA, y_test)


0.9473684210526315

In [27]:
y_pred = svc_model.predict(x_test_PCA)

In [28]:
print(accuracy_score(y_test, y_pred))

0.9473684210526315


In [18]:
x_train.head(5)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
408,17.99,20.66,117.80,991.7,0.10360,0.13040,0.120100,0.088240,0.1992,0.06069,...,21.080,25.41,138.10,1349.0,0.14820,0.37350,0.33010,0.19740,0.3060,0.08503
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.198000,0.104300,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.40000,0.16250,0.2364,0.07678
307,9.00,14.40,56.36,246.3,0.07005,0.03116,0.003681,0.003472,0.1788,0.06833,...,9.699,20.07,60.90,285.5,0.09861,0.05232,0.01472,0.01389,0.2991,0.07804
386,12.21,14.09,78.78,462.0,0.08108,0.07823,0.068390,0.025340,0.1646,0.06154,...,13.130,19.29,87.65,529.9,0.10260,0.24310,0.30760,0.09140,0.2677,0.08824
404,12.34,14.95,78.29,469.1,0.08682,0.04571,0.021090,0.020540,0.1571,0.05708,...,13.180,16.85,84.11,533.1,0.10480,0.06744,0.04921,0.04793,0.2298,0.05974


In [19]:
x_train.iloc[:,10:20].head(5)

,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se
408,0.4537,0.8733,3.061,49.810,0.007231,0.027720,0.025090,0.014800,0.01414,0.003336
4,0.7572,0.7813,5.438,94.440,0.011490,0.024610,0.056880,0.018850,0.01756,0.005115
307,0.1746,1.3050,1.144,9.789,0.007389,0.004883,0.003681,0.003472,0.02701,0.002153
386,0.2666,0.8309,2.097,19.960,0.004405,0.030260,0.043440,0.010870,0.01921,0.004622
404,0.3833,0.9078,2.602,30.150,0.007702,0.008491,0.013070,0.010300,0.02970,0.001432
